# Single Particle performance plots


   * [#$p_{T}$ response](#$p_{T}$-response)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools

In [4]:
# %load samples.py
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v80'
version_V8 = version
version_V9 = 'v81'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V9, 'V9'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V9, 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V8, 'V8'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V8, 'V8')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V9, 'V9'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V9, 'V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]


Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist


In [7]:
# %load settings.py
# %load settings.py
# from samples import get_label_dict
import python.selections as selections
import python.collections as collections

import pprint



def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



# === samples =====================================================
samples = []

samples = samples_ele_V8
samples += samples_ele_V9


sample = 'V9'

do_rate = False

# === TP ==========================================================
# tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']
tps = ['DEF', 'HMvDR', 'HMvDRRebin', 'HMvDRsTC', 'HMvDRTrue']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']
tp_select['HMvDRRebin'] = tp_select['DEF']
tp_select['HMvDRsTC'] = tp_select['DEF']
tp_select['HMvDRTrue'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']



event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label


# all_tpsets = {'DEF': 'dRC3d',
#               'DEFCalib': 'NNDR Calib v1',
#               'DEFNC': 'dRC3d + new Th',
#               'HMvDR': 'HistoMaxC3d + dR(layer)',
#               'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
#               'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
#               'EG': 'EG',
#               'TkEG': 'TkEG',
#               'TkEle': 'TkEle',
#               'TkIsoEle': 'TkIsoEle',
#               'L1Trk': 'L1Track'}


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: TT
[EventManager] registering collection: SimTT
[EventManager] registering collection: HgcrocTT
[EventManager] registering collection: WaferTT
[EventManager] registering 

In [8]:
tpsets


{'DEF': 'dRC3d',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRRebin': 'HM+dR(layer) rebin Cl3d ',
 'HMvDRTrue': 'HM+dR(layer) True Cl3d',
 'HMvDRsTC': 'HM+dR(layer) SuperTC Cl3d '}

In [9]:
# tpset_selections.pop('Emv1')
tpset_selections


{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [10]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [11]:
from python.plotters import tp_genmatched_plotters

histo_primitives = pd.DataFrame() 

for plotter in tp_genmatched_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['DEF' 'HMvDRTrue' 'HMvDR' 'HMvDRRebin' 'HMvDRsTC']


In [7]:
files

{}

In [13]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

for tp in tps:
    hplot.cache_histo(classtype=histos.ClusterConeHistos, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_selections)


<__main__.Sample instance at 0x109f21998>
<__main__.Sample instance at 0x109f21998> DEF all GENPt40
<__main__.Sample instance at 0x109f21998> DEF all GENEtaC
<__main__.Sample instance at 0x109f21998> DEF all GENEtaB
<__main__.Sample instance at 0x109f21998> DEF all GENEtaD
<__main__.Sample instance at 0x109f21998> DEF all GENEtaBC
<__main__.Sample instance at 0x109f21998> DEF all GENPt30
<__main__.Sample instance at 0x109f21998> DEF all GENPt20
<__main__.Sample instance at 0x109f21998> DEF all GENEtaBCD
<__main__.Sample instance at 0x109f21998> DEF all GENPt10
<__main__.Sample instance at 0x109f21998> DEF all nomatch
<__main__.Sample instance at 0x109f21998> DEF all GEN
<__main__.Sample instance at 0x109f21998> DEF Pt10 GENPt40
<__main__.Sample instance at 0x109f21998> DEF Pt10 GENEtaC
<__main__.Sample instance at 0x109f21998> DEF Pt10 GENEtaB
<__main__.Sample instance at 0x109f21998> DEF Pt10 GENEtaD
<__main__.Sample instance at 0x109f21998> DEF Pt10 GENEtaBC
<__main__.Sample instance

<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENEtaD
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENEtaBC
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENPt30
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENPt20
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENEtaBCD
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GENPt10
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 nomatch
<__main__.Sample instance at 0x11b3addd0> DEF EmPt10 GEN
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENPt40
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENEtaC
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENEtaB
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENEtaD
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENEtaBC
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENPt30
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENPt20
<__main__.Sample instance at 0x11b3addd0> DEF EmPt20 GENEtaBCD
<__main__.Sample insta

<__main__.Sample instance at 0x109b56710> DEF Pt10 nomatch
<__main__.Sample instance at 0x109b56710> DEF Pt10 GEN
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENPt40
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENEtaC
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENEtaB
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENEtaD
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENEtaBC
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENPt30
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENPt20
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENEtaBCD
<__main__.Sample instance at 0x109b56710> DEF Pt20 GENPt10
<__main__.Sample instance at 0x109b56710> DEF Pt20 nomatch
<__main__.Sample instance at 0x109b56710> DEF Pt20 GEN
<__main__.Sample instance at 0x109b56710> DEF Pt30 GENPt40
<__main__.Sample instance at 0x109b56710> DEF Pt30 GENEtaC
<__main__.Sample instance at 0x109b56710> DEF Pt30 GENEtaB
<__main__.Sample instance at 0x109b56710> DEF Pt30 GENEtaD
<_

<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GENEtaBC
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GENPt30
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GENPt20
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GENEtaBCD
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GENPt10
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 nomatch
<__main__.Sample instance at 0x109f21998> HMvDR EmPt20 GEN
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENPt40
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENEtaC
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENEtaB
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENEtaD
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENEtaBC
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENPt30
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENPt20
<__main__.Sample instance at 0x109f21998> HMvDR EmPt30 GENEtaBCD
<__main__.Sample instance at 0x109f21998> HMvDR EmPt3

<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt20 GENEtaBCD
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt20 GENPt10
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt20 nomatch
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt20 GEN
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENPt40
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENEtaC
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENEtaB
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENEtaD
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENEtaBC
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENPt30
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENPt20
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENEtaBCD
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GENPt10
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 nomatch
<__main__.Sample instance at 0x11b3adcf8> HMvDR Pt30 GEN
<__main__.Sample instance at 0x11b3adcf8> HMvDR Em GENPt40
<__main__.Sample instance at 

...

<__main__.Sample instance at 0x109f21998> HMvDRsTC EmPt30 GENPt10
<__main__.Sample instance at 0x109f21998> HMvDRsTC EmPt30 nomatch
<__main__.Sample instance at 0x109f21998> HMvDRsTC EmPt30 GEN
<__main__.Sample instance at 0x11b3addd0>
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENPt40
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENEtaC
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENEtaB
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENEtaD
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENEtaBC
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENPt30
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENPt20
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENEtaBCD
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GENPt10
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all nomatch
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC all GEN
<__main__.Sample instance at 0x11b3addd0> HMvDRsTC Pt10 GENPt40
<__main_

<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Pt30 GEN
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENPt40
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENEtaC
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENEtaB
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENEtaD
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENEtaBC
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENPt30
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENPt20
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENEtaBCD
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GENPt10
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em nomatch
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC Em GEN
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC EmPt10 GENPt40
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC EmPt10 GENEtaC
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC EmPt10 GENEtaB
<__main__.Sample instance at 0x11b3adcf8> HMvDRsTC EmPt10 GEN

<__main__.Sample instance at 0x109f21998> HMvDRTrue all GENPt30
<__main__.Sample instance at 0x109f21998> HMvDRTrue all GENPt20
<__main__.Sample instance at 0x109f21998> HMvDRTrue all GENEtaBCD
<__main__.Sample instance at 0x109f21998> HMvDRTrue all GENPt10
<__main__.Sample instance at 0x109f21998> HMvDRTrue all nomatch
<__main__.Sample instance at 0x109f21998> HMvDRTrue all GEN
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENPt40
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENEtaC
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENEtaB
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENEtaD
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENEtaBC
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENPt30
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENPt20
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENEtaBCD
<__main__.Sample instance at 0x109f21998> HMvDRTrue Pt10 GENPt10
<__main__.Sample instance at 0

<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENEtaC
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENEtaB
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENEtaD
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENEtaBC
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENPt30
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENPt20
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENEtaBCD
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GENPt10
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 nomatch
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt20 GEN
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt30 GENPt40
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt30 GENEtaC
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt30 GENEtaB
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt30 GENEtaD
<__main__.Sample instance at 0x11b3addd0> HMvDRTrue EmPt30 GENE

<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 GENPt30
<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 GENPt20
<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 GENEtaBCD
<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 GENPt10
<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 nomatch
<__main__.Sample instance at 0x109b56710> HMvDRTrue Pt30 GEN
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENPt40
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENEtaC
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENEtaB
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENEtaD
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENEtaBC
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENPt30
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENPt20
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENEtaBCD
<__main__.Sample instance at 0x109b56710> HMvDRTrue Em GENPt10
<__main__.Sample instance at 0x109b56710> 

In [14]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V8,PU0,DEF,all,GENPt40,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e96e7a0>
1,V8,PU0,DEF,all,GENEtaC,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e960830>
2,V8,PU0,DEF,all,GENEtaB,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11b3dc908>
3,V8,PU0,DEF,all,GENEtaD,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e96e9e0>
4,V8,PU0,DEF,all,GENEtaBC,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11b476dd0>
5,V8,PU0,DEF,all,GENPt30,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e96c8c0>
6,V8,PU0,DEF,all,GENPt20,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e969ab8>
7,V8,PU0,DEF,all,GENEtaBCD,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e96e098>
8,V8,PU0,DEF,all,GENPt10,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e960d40>
9,V8,PU0,DEF,all,nomatch,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11e960440>


In [15]:
hplot.labels_dict

{'DEF': 'dRC3d',
 'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRRebin': 'HM+dR(layer) rebin Cl3d ',
 'HMvDRTrue': 'HM+dR(layer) True Cl3d',
 'HMvDRsTC': 'HM+dR(layer) SuperTC Cl3d ',
 'PU0': 'PU0',
 'PU200': 'PU200',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'V8': 'V8',
 'V9': 'V9',
 'all': '',
 'nomatch': ''}

## Electron samples

In [18]:

for pu in ['PU0', 'PU200']:
    for tp_sel in ['all']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, pu, ['DEF', 'HMvDR', 'HMvDRRebin'], tp_sel, gen_sel)            
            draw([his.h_n for his in hsets], labels, norm=True, text=text, logy=True, do_stats=False, options='hist')
    #             print hsets[0].h_n.GetBinContent(1)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log s

In [20]:

for pu in ['PU0', 'PU200']:
    for tp_sel in ['all']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, pu, ['DEF', 'HMvDR', 'HMvDRRebin'], tp_sel, gen_sel)            
            draw([his.h_ptRel for his in hsets], labels, norm=True, text=text, logy=True, do_stats=False, options='hist')
    #             print hsets[0].h_n.GetBinContent(1)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log s

In [19]:
for gen_sel in gen_select:
    for tp in tps:
        for tp_sel in tp_select[tp]:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRel for his in hsets], labels, norm=True, text=text, options='hist')


TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: HMvDRRebin_Em_GENEtaD_ptRel


In [27]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaEta for his in hsets], labels, norm=True, text=text)


In [28]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhi for his in hsets], labels, norm=True, text=text)


In [29]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhiVq for his in hsets], labels, text=text, options='COLZ', min_y=-1, max_y=2)


## Photon samples

In [40]:
sample = 'photon'

In [42]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_n for his in hsets], labels, norm=False, text=text, logy=True)


[<ROOT.TH1I object ("DEF_all_GEN_n") at 0x7ff44e3a4290>]
[<ROOT.TH1I object ("DEF_all_GENEtaB_n") at 0x7ff44e4e0a20>]
[<ROOT.TH1I object ("DEF_all_GENEtaBC_n") at 0x7ff44e249750>]
[<ROOT.TH1I object ("DEF_all_GENEtaBCD_n") at 0x7ff44e4eba40>]
[<ROOT.TH1I object ("DEF_all_GENEtaC_n") at 0x7ff44e4f2390>]
[<ROOT.TH1I object ("DEF_all_GENEtaD_n") at 0x7ff44e24f290>]
[<ROOT.TH1I object ("DEF_Em_GEN_n") at 0x7ff44e4f7a90>]
[<ROOT.TH1I object ("DEF_Em_GENEtaB_n") at 0x7ff44e3b0320>]
[<ROOT.TH1I object ("DEF_Em_GENEtaBC_n") at 0x7ff44e3b1d30>]
[<ROOT.TH1I object ("DEF_Em_GENEtaBCD_n") at 0x7ff44e257fd0>]
[<ROOT.TH1I object ("DEF_Em_GENEtaC_n") at 0x7ff44e50c6d0>]
[<ROOT.TH1I object ("DEF_Em_GENEtaD_n") at 0x7ff44e50f510>]
[<ROOT.TH1I object ("DEFMerged_all_GEN_n") at 0x7ff44e518d30>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaB_n") at 0x7ff44e51b8d0>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaBC_n") at 0x7ff44e5210f0>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaBCD_n") at 0x7ff44e3bc5c0>]
[<ROOT.TH

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log s

In [19]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRel for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_ptRel") at 0x7ff44e245240>, <ROOT.TH1F object ("DEF_all_GEN_ptRel") at 0x7ff44e245a80>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_ptRel") at 0x7ff44e4e2370>, <ROOT.TH1F object ("DEF_all_GENEtaB_ptRel") at 0x7ff44e4e2f20>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_ptRel") at 0x7ff44e24a1b0>, <ROOT.TH1F object ("DEF_all_GENEtaBC_ptRel") at 0x7ff44e24ad00>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_ptRel") at 0x7ff44dabe000>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_ptRel") at 0x7ff44dabeab0>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_ptRel") at 0x7ff44e3ab1a0>, <ROOT.TH1F object ("DEF_all_GENEtaC_ptRel") at 0x7ff44e3abd50>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_ptRel") at 0x7ff44e4f3c60>, <ROOT.TH1F object ("DEF_all_GENEtaD_ptRel") at 0x7ff44e4f4380>]
[<ROOT.TH1F object ("DEF_Em_GEN_ptRel") at 0x7ff44dac5240>, <ROOT.TH1F object ("DEF_Em_GEN_ptRel") at 0x7ff44dac5df0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_ptRel") at 0x7ff44e500a30>, <ROOT.TH1F object ("DEF_Em_GENEtaB_ptRel") at

TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: DEFMerged_all_GENEtaC_ptRel


In [45]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaEta for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_deltaEta") at 0x7ff44e76ee60>, <ROOT.TH1F object ("DEF_all_GEN_deltaEta") at 0x7ff44e76dae0>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_deltaEta") at 0x7ff44e5d0480>, <ROOT.TH1F object ("DEF_all_GENEtaB_deltaEta") at 0x7ff44e5d1cb0>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_deltaEta") at 0x7ff44f0563a0>, <ROOT.TH1F object ("DEF_all_GENEtaBC_deltaEta") at 0x7ff44e5d34b0>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaEta") at 0x7ff44e679b90>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaEta") at 0x7ff44e670630>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_deltaEta") at 0x7ff44e686970>, <ROOT.TH1F object ("DEF_all_GENEtaC_deltaEta") at 0x7ff44e6873f0>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_deltaEta") at 0x7ff44e5d23d0>, <ROOT.TH1F object ("DEF_all_GENEtaD_deltaEta") at 0x7ff44e5d4470>]
[<ROOT.TH1F object ("DEF_Em_GEN_deltaEta") at 0x7ff44e772bb0>, <ROOT.TH1F object ("DEF_Em_GEN_deltaEta") at 0x7ff44e772fa0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_deltaEta") at 0x7ff44f053230>, 

In [21]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhi for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_deltaPhi") at 0x7ff44e530670>, <ROOT.TH1F object ("DEF_all_GEN_deltaPhi") at 0x7ff44e530a60>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_deltaPhi") at 0x7ff44e279600>, <ROOT.TH1F object ("DEF_all_GENEtaB_deltaPhi") at 0x7ff44e276560>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_deltaPhi") at 0x7ff44e3c9b20>, <ROOT.TH1F object ("DEF_all_GENEtaBC_deltaPhi") at 0x7ff44e3ca390>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaPhi") at 0x7ff44e532160>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaPhi") at 0x7ff44e532ab0>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_deltaPhi") at 0x7ff44e533a60>, <ROOT.TH1F object ("DEF_all_GENEtaC_deltaPhi") at 0x7ff44e5348e0>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_deltaPhi") at 0x7ff44dadb1a0>, <ROOT.TH1F object ("DEF_all_GENEtaD_deltaPhi") at 0x7ff44dadbbe0>]
[<ROOT.TH1F object ("DEF_Em_GEN_deltaPhi") at 0x7ff44e5359c0>, <ROOT.TH1F object ("DEF_Em_GEN_deltaPhi") at 0x7ff44e536ad0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_deltaPhi") at 0x7ff44e3cbe60>, 

TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: DEFMerged_all_GENEtaC_deltaPhi


In [22]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhiVq for his in hsets], labels, text=text, options='COLZ', min_y=-1, max_y=2)


[<ROOT.TH2F object ("DEF_all_GEN_deltaPhiVq") at 0x7ff44e968400>, <ROOT.TH2F object ("DEF_all_GEN_deltaPhiVq") at 0x7ff44ccb2200>]
[<ROOT.TH2F object ("DEF_all_GENEtaB_deltaPhiVq") at 0x7ff449a7ee00>, <ROOT.TH2F object ("DEF_all_GENEtaB_deltaPhiVq") at 0x7ff44cb4c800>]
[<ROOT.TH2F object ("DEF_all_GENEtaBC_deltaPhiVq") at 0x7ff44cb4ac00>, <ROOT.TH2F object ("DEF_all_GENEtaBC_deltaPhiVq") at 0x7ff4494a4e00>]
[<ROOT.TH2F object ("DEF_all_GENEtaBCD_deltaPhiVq") at 0x7ff44e9e2200>, <ROOT.TH2F object ("DEF_all_GENEtaBCD_deltaPhiVq") at 0x7ff44ea5dc00>]
[<ROOT.TH2F object ("DEF_all_GENEtaC_deltaPhiVq") at 0x7ff44e9e3000>, <ROOT.TH2F object ("DEF_all_GENEtaC_deltaPhiVq") at 0x7ff44ea0d000>]
[<ROOT.TH2F object ("DEF_all_GENEtaD_deltaPhiVq") at 0x7ff44aaea000>, <ROOT.TH2F object ("DEF_all_GENEtaD_deltaPhiVq") at 0x7ff449533400>]
[<ROOT.TH2F object ("DEF_Em_GEN_deltaPhiVq") at 0x7ff44cb4a600>, <ROOT.TH2F object ("DEF_Em_GEN_deltaPhiVq") at 0x7ff44ccc5600>]
[<ROOT.TH2F object ("DEF_Em_GENEtaB_del